# Spoof Attack Analysis


In [1]:
import re
from typing import Dict, Any

import wandb
import polars as pl
import altair as alt

In [2]:
def parse_sweep_url(url: str):
    m = re.search(r'wandb\.ai/([^/]+)/([^/]+)/sweeps/([a-z0-9]+)', url)
    if not m:
        raise ValueError('Unrecognized sweep URL; expected .../ENTITY/PROJECT/sweeps/ID')
    return m.group(1), m.group(2), m.group(3)

def fetch_sweep_runs(entity: str, project: str, sweep_id: str) -> pl.DataFrame:
    """
    Returns a DataFrame where each row is a run.
    Columns include run metadata plus flattened `config.*` and `summary.*` fields.
    """

    api = wandb.Api(timeout=300)
    sw = api.sweep(f'{entity}/{project}/{sweep_id}')
    runs = list(sw.runs)
    records = []
    for r in runs:
        cfg: Dict[str, Any] = dict(getattr(r, 'config', {}) or {})
        summ: Dict[str, Any] = dict(getattr(r, 'summary', {}) or {})
        rec: Dict[str, Any] = {
            'run_id': getattr(r, 'id', None),
            'run_name': getattr(r, 'name', None),
            'state': getattr(r, 'state', None),
            'created_at': str(getattr(r, 'created_at', '')),
            'entity': entity,
            'project': project,
            'sweep_id': sweep_id,
        }

        items = [*cfg.items(), *summ.items()]

        for k, v in items:
            rec[k] = v

        records.append(rec)
    df = pl.DataFrame(records)
    return df


In [18]:
WANDB_ENTITY = "juanbelieni-lab"
WANDB_PROJECT = "watermark-spoof"
SWEEP_ID = "cijomih4"

# Fetch runs into a DataFrame
runs_df = fetch_sweep_runs(WANDB_ENTITY, WANDB_PROJECT, SWEEP_ID)
print(f'Loaded {len(runs_df)} runs from sweep.')
runs_df.head()


Loaded 220 runs from sweep.


run_id,run_name,state,created_at,entity,project,sweep_id,seed,clip_c,model_id,delta_att,num_prompts,window_size,limit_samples,max_new_tokens,_runtime,_step,_timestamp,_wandb,success_rate,z_avg
str,str,str,str,str,str,str,i64,i64,str,f64,i64,i64,i64,i64,i64,i64,f64,object,f64,f64
"""s703d22t""","""bumbling-sweep-1""","""finished""","""2025-08-28T10:33:30Z""","""juanbelieni-lab""","""watermark-spoof""","""cijomih4""",0,2,"""meta-llama/Llama-3.2-3B-Instru…",2.5,100,2,5,500,76,0,1.7564e9,{'runtime': 76},0.07,0.127953
"""k9xv1jar""","""generous-sweep-2""","""finished""","""2025-08-28T10:34:53Z""","""juanbelieni-lab""","""watermark-spoof""","""cijomih4""",1,2,"""meta-llama/Llama-3.2-3B-Instru…",2.5,100,2,5,500,75,0,1.7564e9,{'runtime': 75},0.06,0.11922
"""lafcv9di""","""icy-sweep-3""","""finished""","""2025-08-28T10:36:16Z""","""juanbelieni-lab""","""watermark-spoof""","""cijomih4""",2,2,"""meta-llama/Llama-3.2-3B-Instru…",2.5,100,2,5,500,75,0,1.7564e9,{'runtime': 75},0.08,0.091219
"""hcgblzi4""","""skilled-sweep-4""","""finished""","""2025-08-28T10:37:39Z""","""juanbelieni-lab""","""watermark-spoof""","""cijomih4""",3,2,"""meta-llama/Llama-3.2-3B-Instru…",2.5,100,2,5,500,77,0,1.7564e9,{'runtime': 77},0.03,0.056921
"""49prz5rs""","""trim-sweep-5""","""finished""","""2025-08-28T10:39:07Z""","""juanbelieni-lab""","""watermark-spoof""","""cijomih4""",4,2,"""meta-llama/Llama-3.2-3B-Instru…",2.5,100,2,5,500,77,0,1.7564e9,{'runtime': 77},0.06,0.030289


In [21]:
agg_df = (
    runs_df
    .with_columns((pl.col("limit_samples") * 3).alias("limit_samples"))
    .group_by(["limit_samples", "delta_att"])
    .agg(
      pl.col("success_rate").mean().alias("success_rate"),
      pl.col("z_avg").mean().alias("z_avg"),
    )
    .sort(["delta_att", "limit_samples"])
)

chart = alt.Chart(agg_df.to_pandas()).mark_line(interpolate="monotone").encode(
    x=alt.X("limit_samples", type="quantitative").scale(type="log"),
    y=alt.Y("success_rate", type="quantitative"),
    color=alt.Color("delta_att", type="nominal", title="delta_att"),
)
chart

alt.Chart(...)

In [ ]:
agg_df.write_csv("../web/graph_data.csv")